# Data Analysis

#### Imports

In [ ]:
%matplotlib inline

import pandas as pd
pd.options.display.max_rows = 999
import sqlite3
import statsmodels.formula.api as sm
# import statsmodels.api as s
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics

# from sklearn.linear_model import LogisticRegression
# from sklearn.cross_validation import cross_val_score

#### Database Connection

In [ ]:
conn = sqlite3.connect("./sample.db")

#### Occupy Dataframe

In [ ]:
# df_occupy = pd.read_sql(sql="SELECT * FROM occupy", con=conn)
df_occupy_1 = pd.read_sql(sql="SELECT room, date, time, associated_client_count, authenticated_client_count FROM occupy", con=conn)
df_occupy_2 = pd.read_sql(sql="SELECT room, date, time, module_code, occupancy FROM occupy", con=conn)

#### Only consider rows with the following conditions satisfied:

* **Condition 1**: time between 9:00 and 17:00
* **Condition 2:** This condition is because we want to take the average authenticated client count between these times. We are disregarding the other figures because we are not sure of the DHCP lease duration. For instance, a number of the devices could still be being considered from an individual who has already left the room but the lease is still considered.
    - time >= quarter past the hour
    - time <= quarter past the hour

In [ ]:
df_occupy_1["condition1"] = df_occupy_1["time"].apply(lambda x: "true" if 16 >= int(x[0:2]) >= 9 else "false")
df_occupy_1 = df_occupy_1[df_occupy_1["condition1"] == "true"]
df_occupy_1

In [ ]:
df_occupy_1["condition2"] = df_occupy_1["time"].apply(lambda x: "true" if 45 >= int(x[-2:]) >= 15 else "false")
df_occupy_1 = df_occupy_1[df_occupy_1["condition2"] == "true"]
df_occupy_1

In [ ]:
del df_occupy_1["condition1"]
del df_occupy_1["condition2"]
df_occupy_1["time"] = df_occupy_1["time"].apply(lambda x: x[0:3] + "00")
df_occupy_1

In [ ]:
df_occupy_2

In [ ]:
df_occupy_merge = pd.merge(left = df_occupy_1, right = df_occupy_2, how="outer", on=["room", "date", "time"]) 
df_occupy_merge.head(10)

In [ ]:
df_occupy_merge.tail(10)

In [ ]:
df_occupy = df_occupy_merge.groupby(["room", "date", "time", "module_code"], as_index=False).mean()

In [ ]:
# df_occupy["authenticated_client_count"] = df_occupy["authenticated_client_count"].round(0)
# df_occupy["associated_client_count"] = df_occupy["associated_client_count"].round(0)
print(df_occupy.shape)
df_occupy

#### Remove rows with NaN values 

In [ ]:
df_occupy = df_occupy.dropna() 
print(df_occupy.shape)
df_occupy

#### Module Dataframe

In [ ]:
df_module = pd.read_sql(sql="SELECT * FROM module", con=conn)
df_module.head(10)

In [ ]:
df_module.tail(10)

#### Location Dataframe

In [ ]:
df_location = pd.read_sql(sql="SELECT * FROM location", con=conn)
df_location.head(10)

In [ ]:
df_location.tail(10)

#### Close db connection

In [ ]:
conn.close()

## Construct Analytics Base Table

#### Join tables

In [ ]:
df_abt = pd.merge(left = df_occupy, right = df_module, how="outer", on=["module_code"]) 
df_abt = pd.merge(left = df_abt, right = df_location, how="outer", on=["room"]) 
print(df_abt.shape)
df_abt

In [ ]:
df_abt["occupancy_number"] = df_abt["occupancy"] * df_abt["capacity"]

#### Test: Does ground truth data have more students than what is registered

In [ ]:
df_abt["reg_students_less_occ"] = df_abt["reg_students"] - df_abt["occupancy_number"]

In [ ]:
errors = df_abt[df_abt["reg_students_less_occ"] < 0]
print(errors.shape)
errors[["reg_students_less_occ", "occupancy_number", "reg_students"]]

* For those rows with -10, change occupancy_number to the number of registered students
* Should we disregard rows with greater difference because data should be considered to be an outlier
* Also, what to do with the rows that have 0 registered students i.e. was there a class? If not should we even consider it?......

#### Get % error 

In [ ]:
df_abt["error_perc"] = (df_abt["reg_students"] - df_abt["occupancy_number"]) / df_abt["capacity"]
df_abt

#### Remove those with an error larger than 10%

In [ ]:
df_abt = df_abt[df_abt["error_perc"] >= -0.1]
print(df_abt.shape)
df_abt

#### Those with an error equal to or less than a 10% error, adjust figure to equal ground truth

In [ ]:
df_abt["adjustment"] = df_abt["reg_students_less_occ"].apply(lambda x: x if x <= 0 else 0)
df_abt

In [ ]:
df_abt["occupancy_number_adj"] = df_abt["adjustment"] + df_abt["occupancy_number"]

#### Analysis of data with occupancy number equal to zero (NEED TO DO SOMETHING WITH THIS!!!!)

In [ ]:
a = df_abt[df_abt["occupancy_number_adj"] == 0]
a[["module_code", "reg_students", "occupancy_number_adj", "authenticated_client_count"]]

#### Create binned associated and authenticated client counts for logistic regression

In [ ]:
# def bin(r):
#     '''Bin client counts into percintle categories'''
#     if r < .125:
#         return .0
#     elif r < .375: 
#         return .25
#     elif r < .625:
#         return .5
#     elif r < .875:
#         return .75
#     else:
#         return 1.0 

# df_abt["assoc_binned"] = df_abt["associated_client_count"] / df_abt["capacity"]
# df_abt["assoc_binned"] = df_abt["assoc_binned"].apply(lambda x: bin(x))
# df_abt["auth_binned"] = df_abt["authenticated_client_count"] / df_abt["capacity"]
# df_abt["auth_binned"] = df_abt["auth_binned"].apply(lambda x: bin(x))
# df_abt

#### Regression dataframe

In [ ]:
df_regression = df_abt[["occupancy_number_adj", "authenticated_client_count"]]
df_regression

#### Normalize Features and remove outliers (?)

In [ ]:
# def normalize(df, feature):
#     return (df[feature] - df[feature].mean()) / df[feature].std()

# for feature in normFeat:
#     df_new[feature] = normalize(df_new, feature)

# def removeOutliers(df, feature):
#     '''Remove outliers (more than 3 std devs from mean)'''
#     return df[np.abs(df[feature] - df[feature].mean()) <= (3 * df[feature].std())]

# for feature in normFeat:
#     df_new = removeOutliers(df_new, feature)

### Feature Analysis

In [ ]:
colour = dict(boxes='DarkGreen', whiskers='DarkOrange', medians='DarkBlue', caps='Gray') #colours for box plots

plt.figure() 

plt.subplot(211).set_title("occupancy_number_adj", fontsize=15)
df_regression["occupancy_number_adj"].plot(kind = 'box', figsize = (15,15), color=colour, sym='r+') 

plt.subplot(212).set_title("authenticated_client_count", fontsize=15)
df_regression["authenticated_client_count"].plot(kind = 'box', figsize = (15,15), color=colour, sym='r+') 

plt.tight_layout(pad=2.0, w_pad=2.0, h_pad=3.0) #set padding
# plt.savefig("Box_Plots.png")

In [ ]:
print(df_regression.dtypes)

corrMatrix = df_regression.corr()
corrMatrix

In [ ]:
plt.figure() 
df_regression.plot(kind="scatter", x="authenticated_client_count", y="occupancy_number_adj", label="%.3f" % corrMatrix.as_matrix()[0,1], figsize=(20, 6))
# plt.savefig("Scatter_Plot.png")

### Regression

#### Statsmodels

In [ ]:
lm_sm = sm.ols(formula="occupancy_number_adj ~ authenticated_client_count - 1", data=df_regression).fit()
print(lm_sm.summary())

#### Scikit Learn

In [ ]:
X = df_regression["authenticated_client_count"].reshape(len(df_regression["authenticated_client_count"]), 1)
y = df_regression["occupancy_number_adj"]

lm_scikit = LinearRegression(fit_intercept=False)

# Fit the model 
lm_scikit.fit(X, y)

# Coefficients
print("Coefficients: \n", lm_scikit.coef_)
# The intercept
print('Intercept: \n', lm_scikit.intercept_)
# The mean square error: MSE is more popular than MAE because MSE "punishes" larger errors. 
# But, RMSE is even more popular than MSE because RMSE is interpretable in the "y" units.
print("Residual sum of squares: %.2f" % np.mean((lm_scikit.predict(X) - y) ** 2))
print(metrics.mean_squared_error(y, lm_scikit.predict(X)))
print(np.sqrt(metrics.mean_squared_error(y, lm_scikit.predict(X))))

# Explained variance score: coefficient of determination (R^2)
print("Variance score: %.2f" % lm_scikit.score(X, y))

# Plot outputs
plt.scatter(X, y,  color='navy')
plt.plot(X, lm_scikit.predict(X), color='red',linewidth=2)
plt.figure(figsize=(20,20))
plt.show()

In [ ]:
print(lm_scikit.predict(X))

In [ ]:
df_regression["predicted_occupancy"] = pd.Series(lm_scikit.predict(X), index=df_regression.index)
df_regression

### Train & Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Train the model using the training sets
lm_scikit.fit(X_train, y_train)

# The coefficients
print('Coefficients: \n', lm_scikit.coef_)
# The intercept
print('Intercept: \n', lm_scikit.intercept_)
# The mean square error
print("Residual sum of squares: %.2f" % np.mean((lm_scikit.predict(X_test) - y_test) ** 2))
print(metrics.mean_squared_error(y_test, lm_scikit.predict(X_test)))
print(np.sqrt(metrics.mean_squared_error(y_test, lm_scikit.predict(X_test))))
# Explained variance score: coefficient of determination (R^2)
print('Variance score: %.2f' % lm_scikit.score(X_test, y_test))

# Plot outputs
plt.scatter(X_test, y_test,  color='navy')
plt.plot(X_test, lm_scikit.predict(X_test), color='red',linewidth=3)
plt.figure(figsize=(20,20))
plt.show()

In [ ]:
# plt.scatter(lm_scikit.predict(X_train), lm_scikit.predict(X_train) - y_train, color="navy", s=40, alpha=0.5)
# plt.scatter(lm_scikit.predict(X_test), lm_scikit.predict(X_test) - y_test, color="green", s=40)
# plt.hlines(y=0, xmin=0, xmax=160)
# plt.title("Residual Plot: training (Navy) and test (Green) data")
# plt.ylabel("Residuals")

#### Construct a day column

In [ ]:
# def get_day(date_int):
#     """Takes date int in format yyyymmdd and returns weekday string.
    
#     Uses datetime.date"""
    
#     weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
#     year = date_int // 10000
#     month = (date_int % 10000) // 100
#     day = date_int % 100
    
#     # return index of weekdays list, which is the correct weekday. Uses datetime.date
#     try:
#         return weekdays[date(year, month, day).weekday()]
#     except ValueError:
#         return "Incorrect number of days/months"
    
# def add_day(df):
#     """Adds Day column to df.
    
#     Uses the int date column to find the weekday it refers to."""
    
#     # copy df so it isn't overwritten
#     df_new = df.copy(deep=True)
    
#     # get day from date, add to df
#     if 'day' not in df_new.columns:
#         days = []
#         for i in range(len(df_new)):
#             days.append(get_day(df_new.date.iloc[i]))
#         df_new["day"] = days
#     else:
#         for i in range(len(df_new)):
#             if pd.isnull(df_new.day.iloc[i]):
#                 df_new["day"].iloc[i] = get_day(df_new.date.iloc[i])
#     return df_new